In [40]:
#this code only works for hashtags; most keywords that you would search have too many results for this code to run

from congfig import * 
import pandas as pd
import tweepy
import time
import numpy as np

In [62]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
#swap these for your information, found on twitter api

#API
consumer_key = API_KEY
#API Secret
consumer_secret = API_SECRET
#Token
access_key = ACCESS_TOKEN
#Token Secret
access_secret= ACCESS_SECRET
#Bearer
bearer_token = BEARER

In [42]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
#swap for your bearer token
client = tweepy.Client(bearer_token)

In [37]:
#test to see if you can scrape

query = ['#Springfield #kilograms']
start_time ='2017-01-01T00:00:00Z'
end_time = '2022-04-01T00:00:00Z'
for tweet in tweepy.Paginator(client.search_all_tweets, query=query,tweet_fields = ['context_annotations', 'created_at'], start_time=start_time, end_time=end_time, max_results=100).flatten(limit=1000):
    print(str(tweet.text) + str(tweet.created_at))

RT @CraigEpstein18: As he prepares for his sophomore season with #RFootball, @HamiltonKyonte talks spring ball and the transition from #RU…2022-03-28 15:35:08+00:00
RT @CraigEpstein18: As he prepares for his sophomore season with #RFootball, @HamiltonKyonte talks spring ball and the transition from #RU…2022-03-28 15:34:26+00:00
As he prepares for his sophomore season with #RFootball, @HamiltonKyonte talks spring ball and the transition from #RU wrestling. #Rutgers @RutgersRivals https://t.co/NPIsHAQo5v2022-03-28 15:28:19+00:00
RT @RutgersRivals: .@HamiltonKyonte talks #Rutgers Football transition from #RU Wrestling and more as he prepares for year two on the banks…2022-03-28 15:22:22+00:00
.@HamiltonKyonte talks #Rutgers Football transition from #RU Wrestling and more as he prepares for year two on the banks. 

https://t.co/VCY0Th2L5B2022-03-28 12:42:55+00:00
RT @NameOfMy1stBand: Happening today! At 3:30 PM 🎸 Tix 10$ at the Rutgers Student Center 126 College Ave @RutgersU @Screamales @

TooManyRequests: 429 Too Many Requests
Too Many Requests

In [168]:
hashtags = pd.read_csv('Hashtags.csv', header=0)
location = list(hashtags[~hashtags['1'].isna()]['1'])
content = list(hashtags[~hashtags['2'].isna()]['2'])

In [169]:
#all combos of hashtags

hitList = ['#'+str(x)+' '+'#'+str(y) for x in location for y in content]

In [170]:
print(len(location), len(content), len(hitList))

82 254 20828


In [179]:
#clean out wildcard characters
import re
for s in hitList:
    if re.findall('[(+*)]',s):
        hitList.remove(s)
for s in hitList:
    if re.findall('[(+*)]',s):
        print(s)     
for s in hitList:
    if re.findall('[(&)]',s):
        hitList.remove(s)
for s in hitList:
    if re.findall('[(&)]',s):
        print(s)     

In [159]:
#change the numbers in brackets to manually iterate through the hashtag list
#searching everything at once results in too many requests

#run all blocks here and down for each partial scrape. the only things that need to be changed each time 
#...scraping a chunk of hashtags are the numbers here and the filename to save

hitList10 = hitList[0:40]
print(hitList10)
print(len(hitList10))

['#Syracuse #boylove', '#Syracuse #BLogo ', '#Syracuse #CLogo', '#Syracuse #GLogo', '#Syracuse #kiddie', '#Syracuse #little', '#Syracuse #teacup', '#Syracuse #teen', '#Syracuse #gayboy', '#Syracuse #tradeddropbox', '#Syracuse #young', '#Syracuse #trade', '#Syracuse #trading', '#Syracuse #girllove', '#Syracuse #Childlove', '#Syracuse #dropboxlinktrade', '#Syracuse #mega', '#Syracuse #megalink', '#Syracuse #lolita', '#Syracuse #twink', '#Syracuse #Prepago', '#Syracuse #student', '#Syracuse #sissy', '#Syracuse #petite', '#Syracuse #ladyboy', '#Syracuse #eros', '#Syracuse #Chad ', '#Syracuse #dating', '#Syracuse #meetup', '#Syracuse #mistress', '#Syracuse #dinnerdate', '#Syracuse #Escort', '#Syracuse #escort', '#Syracuse #escortads', '#Syracuse #luxurycompanion', '#Syracuse #skipthegames', '#Syracuse #virtualdates', '#Syracuse #flyme2you', '#Syracuse #flymetoyou', '#Syracuse #cityhotties']
40


In [181]:
#need to run this each time you update hitlist10 so that it updates in the scraper
def divide_chuncks(l,n):
    for i in range(0,len(l),n):
        yield l[i:i+n]
matrix= list(divide_chuncks(hitList,40))
ht1 = matrix
date_since = '2017-01-01'
numtweet = 1000

In [49]:
#this is the scraper code, taken from the original manual scraper we used

def scrape(ht1, date_since, numtweet):
 
        # Creating DataFrame using pandas
        #db1 is intermediary dataframe
        db1 = pd.DataFrame(columns = ['text','date'])
 
        # We are using .Cursor() to search
        # through twitter for the required tweets.
        # The number of tweets can be
        # restricted using .items(number of tweets)
        
        tweets = tweepy.Paginator(client.search_all_tweets, query=ht1, tweet_fields = ['context_annotations', 'created_at'], start_time=start_time, max_results=100).flatten(limit=1000)
            
 
        # .Cursor() returns an iterable object. Each item in
        # the iterator has various attributes
        # that you can access to
        # get information about each tweet
        list_tweets = [tweet for tweet in tweets]
 
        # Counter to maintain Tweet Count
        i = 1
 
        # we will iterate over each tweet in the
        # list for extracting information about each tweet
        for tweet in list_tweets:
                text = tweet.text
                date = tweet.created_at
                #description = tweet.user.description
                #followers = tweet.followers_count
                #hashtags = tweet.entities['hashtags']
 
                # Retweets can be distinguished by
                # a retweeted_status attribute,
                # in case it is an invalid reference,
                # except block will be executed
                
                
                #hashtext = list()
                #for j in range(0, len(hashtags)):
                 #      hashtext.append(hashtags[j]['text'])
                
                # Here we are appending all the
                # extracted information in the DataFrame
                ith_tweet = [text, date]
                db.loc[len(db)] = ith_tweet
 
                # Function call to print tweet data on screen
                #printtweetdata(i, ith_tweet)
                i = i+1
        #db is master dataframe
        db.append(db1)

In [65]:
db = pd.DataFrame(columns=['text','date'])

In [138]:
class getRandom:
    def __init__(self,l):
        self.l = l
        self.processed = []
    def getRandomSample(self):
        self.idx = np.random.randint(0,len(self.l))
        self.sample = self.l.pop(self.idx)
        self.processed.append(self.l.pop())
        return self.sample
    def returnProcessed(self):
        return self.processed
    def returnProcessedLength(self):
        return len(self.processed)
    def __iter__(self):
        yield from self.l
    def addToProcessed(self,ext):
        self.processed.extend(ext)
    def __len__(self):
        return len(self.l)
        

In [182]:
#remove processed records so far
mat = matrix[37:].copy()
#initalize random class
randomVal = getRandom(mat)
# add in already processed records
randomVal.addToProcessed(matrix[:37])

In [194]:
#iterates through list for scraping
#this counts to the length of ht1 while scraping then keyboardinterrupts
#"toomanyrequests" prints between numbers & can print multiple times. this is correct
#if "toomanyrequests" prints without numbers, wait 15 minutes, and try scraping again
#if db is empty but numbers are printed until len(hitList10), there are no tweets in this set of pairs


for i in range(len(randomVal)):
    m = randomVal.getRandomSample()
    c=0
    start=-1
    waiting =False
    print('Matrix:')
    print(m)
    
    while c <= len(m)-1:
        if waiting:
            if (time.time() - start) >= 1:
                waiting = False
                start = time.time()
        else:
            try: 
                scrape(m[c], date_since, numtweet)
                c += 1
            except tweepy.errors.TooManyRequests:
                waiting = True
                start = time.time()
    print('DB Len:', len(db))

Matrix:
['#Toga #SplendaDaddy', '#Toga #SplendaMama', '#Toga #SplendaMommy', '#Toga #LTA', '#Toga #NSA', '#Toga #ORO', '#Toga #PPM', '#Toga #hookup', '#Toga #POT', '#Toga #SA', '#Toga #STA', '#Toga #paypig', '#Toga #SB', '#Toga #RoadSugar', '#Toga #TravelingArrangement', '#Toga #SugarOnTheSide', '#Toga #bear', '#Toga #SD', '#Toga #SBSD', '#Toga #SDSB', '#Toga #PapiDulce', '#Toga #SaltDaddy', '#Toga #SaltMama', '#Toga #SaltMommy', '#Toga #MentoringArrangement', '#Toga #SugarBowl', '#Toga #SM', '#Toga #SR', '#Toga #Companion', '#Toga #companions', '#Toga #Companionship', '#Toga #SugarPup', '#Toga #BigDaddy', '#Toga #SugarBear', '#Toga #SugarDaddiesWanted', '#Toga #SugarMommyWanted', '#Toga #snap', '#Toga #snapchat', '#Binghamton #boylove', '#Binghamton #BLogo ']
DB Len: 17169
Matrix:
['#Schenectady #lingerie', '#Schenectady #Freestyle', '#Schenectady #Daddy ', '#Schenectady #CollegeBaby', '#Schenectady #TuitionBaby', '#Schenectady #DS', '#Schenectady #AngelBaby', '#Schenectady #courtesan

KeyboardInterrupt: 

In [195]:
#outputs all scraped tweets
db

,text,date
0,"To a GREAT Friend &amp; Kind Heart, here are A...",2020-12-07 15:57:29+00:00
1,remember when #Syracuse was a football blue bl...,2020-10-20 22:46:37+00:00
2,#Syracuse #orange #autograph #floyd #little #j...,2020-04-18 08:39:16+00:00
3,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 14:28:29+00:00
4,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 05:15:17+00:00
...,...,...
17291,RT @TramontaneCafe: Hey! #Baking friends:WHY ...,2017-03-08 08:35:07+00:00
17292,RT @TramontaneCafe: Hey! #Baking friends:WHY ...,2017-03-07 21:16:10+00:00
17293,Hey! #Baking friends:WHY FORK LINES IN #PEANU...,2017-03-06 15:54:05+00:00
17294,Rawr!!! Check out these adorable costumes! #Ha...,2020-10-31 23:10:41+00:00


In [196]:
# change csv filename every time you run this cell block
filename = r'data/tenthrun.csv'
db.to_csv(filename)